### cloning the ml zoomcamp

1.  To clone the repo <br><br> `$ git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git`

2. Go into the folder <br><br> `$ cd machine-learning-zoomcamp/cohorts/2025/05-deployment/homework`

3. In terminal 1, build the image using the dockerfile <br><br> `$ docker build -f Dockerfile_full -t zoomcamp-model:3.13.10-hw10 .`

# QUESTION 1 - Serve the app locally and get the predictions

4. run the docker container to serve the app from the docker container <br><br> `$ docker run -it --rm -p 9696:9696 zoomcamp-model:3.13.10-hw10`

If you experience "connect: invalid argument" error when trying to run the container in a virtual machine, try restarting containerd runtime and docker daemon using the following command:
<br><br>
`$ sudo systemctl restart containerd && sudo systemctl restart docker`

5. In terminal #2, run the test to send a request and fetch the response from the containerized app being served <br><br> `$ python3 q6_test.py`

OUTPUT (terminal #1):

`INFO:     Started server process [1] INFO:     Waiting for application startup. INFO:     Application startup complete.`
`INFO:     Uvicorn running on http://0.0.0.0:9696 (Press CTRL+C to quit)`

`INFO:     172.18.0.1:54492 - "POST /predict HTTP/1.1" 200 OK`


OUTPUT (terminal #2):

`{'conversion_probability': 0.49999999999842815, 'conversion': False}`

6. In terminal #1, stop the docker container <br><br> <ctrl+C>

# QUESTION 2 - Setting up `kubectl` and `kind` and finding version of kind

1. find the architecture of the machine <br><br> `$ uname -m`

2. if x86_64, install `kubectl` by running the following: <br><br> `$ curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"`
<br> <br> <br>
Validate the binary: <br> <br>
`$ curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl.sha256"` <br> <br>`$ echo "$(cat kubectl.sha256)  kubectl" | sha256sum --check` <br> <br> `kubectl: OK`

3. install kubectl <br><br> `$ sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl` 
<br><br> `$ chmod +x kubectl`
<br><br> `$ mkdir -p ~/.local/bin`
<br><br> `$ mv ./kubectl ~/.local/bin/kubectl`

4. check if kubectl is successfully installed <br><br> `$ kubectl version --client` <br><br> `Client Version: v1.34.3 Kustomize Version: v5.7.1`

5. if x86_64, install `kind` by running the following:
<br><br> `$ [ $(uname -m) = x86_64 ] && curl -Lo ./kind https://kind.sigs.k8s.io/dl/v0.30.0/kind-linux-amd64`
<br><br> `$ chmod +x ./kind`
<br><br> `$ sudo mv ./kind /usr/local/bin/kind`
<br><br> `$ go install sigs.k8s.io/kind@v0.30.0u`

6. check the kind version <br><br> `$ kind --version` <br><br> `kind version 0.30.0`

# QUESTION 3 & 4 - Smallest deployable computing unit and type of service already running in kubectl

* smallest deployable computing unit that we can create & manage in kubernetes --> Pod

* `kubectl`'s list of running services <br><br> `$ kubectl get services` <br><br> `NAME kubernetes`  <br> `TYPE    ClusterIP`   <br>    `CLUSTER-IP  172.20.0.1` <br>    `EXTERNAL-IP <none>` <br> `PORT(S) 443/TCP` <br> `AGE 59m`

# QUESTION 5 - Register the docker image with kind

1. start a kind cluster <br><br> `$ kind create cluster`


`Creating cluster "kind" ...`
 `✓ Ensuring node image (kindest/node:v1.34.0) 🖼 `
 `✓ Preparing nodes 📦  `
 `✓ Writing configuration 📜 `
 `✓ Starting control-plane 🕹️ `
 `✓ Installing CNI 🔌 `
 `✓ Installing StorageClass 💾 `
`Set kubectl context to "kind-kind"`
`You can now use your cluster with:`
`kubectl cluster-info --context kind-kind`
`Have a nice day! 👋`

2. link kind with kubectl <br><br> `$ kubectl cluster-info --context kind-kind` <br><br> `Kubernetes control plane is running at ... CoreDNS is running at ...`

3. load the docker image to the cluster to register with kind <br><br> `$ kind load docker-image zoomcamp-model:3.13.10-hw10`

# QUESTION 6 - Create a deployment config file and choosing the right port

1. create a deployment.yaml for deployment configuration

In [ ]:
cat << EOF > deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.13.10-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"
            cpu: "200m"
        ports:
        - containerPort: 9696
EOF


2. Applying the deployment.yaml configs <br><br> `$ kubectl apply -f deployment.yaml` 

* the target port used here is `9696`